In [5]:
import requests  # hace llamdas HTTP a la API
import pandas as pd # organiza los datos en tablas
import numpy as np # se utiliza cuando faltan valores
from keys import OWM_key# Importa tu API key

# se define ciudad y país
city = "Busan"
country = "KR"

# construir URL con la key
url = f"http://api.openweathermap.org/data/2.5/forecast?q={city},{country}&appid={OWM_key}&units=metric&lang=en"

# hacer peticion a la API y se guarda la respuesta
response = requests.get(url)

# si la peticion fue exitosa
print("Status code:", response.status_code)

# Procesar datos si la respuesta es correcta
if response.status_code == 200:
    data = response.json()

    # Extraer forecast
    forecast_list = data.get('list', [])

    # Listas para DataFrame
    times, temperatures, humidities = [], [], []
    weather_statuses, wind_speeds, rain_volumes, snow_volumes = [], [], [], []

    for entry in forecast_list:
        times.append(entry.get('dt_txt', np.nan)) # dt_txt → fecha y hora.
        temperatures.append(entry.get('main', {}).get('temp', np.nan)) # main.temp → temperatura.
        humidities.append(entry.get('main', {}).get('humidity', np.nan)) #main.humidity → humedad.
        weather_statuses.append(entry.get('weather', [{}])[0].get('main', np.nan)) #weather[0].main → estado del clima
        wind_speeds.append(entry.get('wind', {}).get('speed', np.nan)) #wind.speed → velocidad del viento.
        rain_volumes.append(entry.get('rain', {}).get('3h', np.nan)) #rain.3h → volumen de lluvia acumulado en 3 horas
        snow_volumes.append(entry.get('snow', {}).get('3h', np.nan)) #snow.3h → volumen de nieve acumulado en 3 horas.
        #Si algo no existe, guarda np.nan en lugar de error

    # Crear DataFrame
    df = pd.DataFrame({
        'time': times,
        'temperature': temperatures,
        'humidity': humidities,
        'weather_status': weather_statuses,
        'wind_speed': wind_speeds,
        'rain_volume_3h': rain_volumes,
        'snow_volume_3h': snow_volumes
    })

    # convierte la columna time en formato fecha
    df['time'] = pd.to_datetime(df['time'])

    print(df.head()) #muestra las primeras 5 filas de la tabla
else:
    print("Error en la petición:", response.status_code)


Status code: 200
                 time  temperature  humidity weather_status  wind_speed  \
0 2025-10-02 00:00:00        18.44        74         Clouds        2.01   
1 2025-10-02 03:00:00        22.51        63         Clouds        1.94   
2 2025-10-02 06:00:00        24.91        58         Clouds        3.28   
3 2025-10-02 09:00:00        23.15        63         Clouds        2.98   
4 2025-10-02 12:00:00        21.35        66           Rain        2.75   

   rain_volume_3h  snow_volume_3h  
0             NaN             NaN  
1             NaN             NaN  
2             NaN             NaN  
3             NaN             NaN  
4            0.25             NaN  
